In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna
import os
import sys

operating_system = 'mac'

if operating_system == 'win':
    os.chdir('C:/Users/fabau/OneDrive/Documents/GitHub/master-project-cleaned/')
elif operating_system == 'curnagl':
    os.chdir('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/cleaner_version/')
else:
    os.chdir('/Users/fabienaugsburger/Documents/GitHub/master-project-cleaned/')

util_perso = os.path.abspath('util/processing')
sys.path.append(util_perso)
util_perso = os.path.abspath('util/gev')
sys.path.append(util_perso)
util_perso = os.path.abspath('util/feature_selection')
sys.path.append(util_perso)

from extraction_squares import split_storm_numbers
from data_processing import depickle
import selection_vars

/Users/fabienaugsburger/anaconda3/envs/master/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
def r2_score_perso(y_true, y_pred):
    #y_mean_train = np.mean(y_true)
    #ss_res_train = np.sum(y_true - y_pred) ** 2
    #ss_tot_train = np.sum(y_true - y_mean_train) ** 2
    #r2_train = 1 - (ss_res_train / ss_tot_train)
    # try:
    #     y_true = y_true.to_numpy().flatten()
    # except:
    #     y_true = y_true.flatten()
    # try:
    #     y_pred = y_pred.to_numpy().flatten()
    # except:
    #     y_pred = y_pred.flatten()

    if np.sum((y_true-np.mean(y_true))**2) == 0:
        print('Warning: y_true and np.mean(y_true) are equal, R2 cannot be calculated')
        r2 = 'nan'
    else:
        r2 = 1 - np.sum((y_true-y_pred)**2)/np.sum((y_true-np.mean(y_true))**2)

    return r2

In [131]:
pred = pd.DataFrame(np.array([[1, 1, 1.9, 1],
                     [1, 1, 1, 1]]))
true = pd.DataFrame(np.array([[1, 1, 2, 1],
                     [1, 1, 1, 1]]))

r2_skl = r2_score(pred, true)
print(r2_skl)
print(r2_score_perso(pred, true))

0.9938271604938271
0.9885714285714285


In [26]:
models = ['random_forest', 'xgboost']
seeds = [42, 1996, 45319, 43709, 19961106, 28012025, 15012025, 2019, 111194, 19052024]
nvars = [20, 30, 40]
output = ['cdf', 'max']


r2_scores = pd.DataFrame(columns=['seed', 'nvar', 'model', 
                                  'r2_train_cdf', 'r2_validation_cdf', 'r2_test_cdf',
                                  'r2_train_cdf_skl', 'r2_validation_cdf_skl', 'r2_test_cdf_skl',
                                  'r2_train_max', 'r2_validation_max', 'r2_test_max',
                                  'r2_train_max_skl', 'r2_validation_max_skl', 'r2_test_max_skl',
                                  'rmse_train_cdf', 'rmse_validation_cdf', 'rmse_test_cdf',
                                  'rmse_train_max', 'rmse_validation_max', 'rmse_test_max'])

for seed in tqdm(seeds):
    for nvar in nvars:
        for model in models:
            # Initialize a dictionary for a single row
            r2_temp = {'seed': seed, 'nvar': nvar, 'model': model}

            for out in output:
                # Load the model
                ml = depickle(f'ml_scripts/new_feature_selection/seed_{seed}/model_{model}/model_{out}_{nvar}.pkl')

                # Load the data
                X_train_original = pd.read_csv(f'ml_scripts/new_feature_selection/seed_{seed}/X_train_{nvar}.csv')
                X_train = X_train_original.to_numpy()
                y_train_original = pd.read_csv(f'ml_scripts/new_feature_selection/seed_{seed}/y_train_{out}.csv')
                y_train = y_train_original
                X_validation = pd.read_csv(f'ml_scripts/new_feature_selection/seed_{seed}/X_validation_{nvar}.csv').to_numpy()
                y_validation = pd.read_csv(f'ml_scripts/new_feature_selection/seed_{seed}/y_validation_{out}.csv')
                X_test = pd.read_csv(f'ml_scripts/new_feature_selection/seed_{seed}/X_test_{nvar}.csv').to_numpy()
                y_test = pd.read_csv(f'ml_scripts/new_feature_selection/seed_{seed}/y_test_{out}.csv')

                # Make predictions and flatten
                y_train_pred = ml.predict(X_train)#.flatten()
                y_train = y_train.to_numpy().flatten()
                print(y_train.shape, y_train_pred.shape)
                y_train_flat = y_train.flatten()
                print('reflat:',y_train_flat.shape)
                y_validation_pred = ml.predict(X_validation).flatten()
                y_validation = y_validation.to_numpy().flatten()
                y_test_pred = ml.predict(X_test).flatten()
                y_test = y_test.to_numpy().flatten()

                # Select the top 5 features
                #top_5_features = selection_vars.feature_selection(X_train_original, X_train, y_train_original, ml)

                # Compute R² scores on the perso version
                r2_train = r2_score_perso(y_train, y_train_pred)
                r2_validation = r2_score_perso(y_validation, y_validation_pred)
                r2_test = r2_score_perso(y_test, y_test_pred)

                # Compute R2 scores on the sklearn version
                r2_train_skl = r2_score(y_train, y_train_pred)
                r2_validation_skl = r2_score(y_validation, y_validation_pred)
                r2_test_skl = r2_score(y_test, y_test_pred)

                # Compute RMSE scores
                rmse_train = np.sqrt(np.mean((y_train - y_train_pred) ** 2))
                rmse_validation = np.sqrt(np.mean((y_validation - y_validation_pred) ** 2))
                rmse_test = np.sqrt(np.mean((y_test - y_test_pred) ** 2))

                # Update the dictionary with the R² values based on `out`
                if out == 'cdf':
                    r2_temp.update({
                        'r2_train_cdf': r2_train,
                        'r2_validation_cdf': r2_validation,
                        'r2_test_cdf': r2_test,
                        'r2_train_cdf_skl': r2_train_skl,
                        'r2_validation_cdf_skl': r2_validation_skl,
                        'r2_test_cdf_skl': r2_test_skl,
                        'rmse_train_cdf': rmse_train,
                        'rmse_validation_cdf': rmse_validation,
                        'rmse_test_cdf': rmse_test,
                        #'top_5_features_cdf': top_5_features
                    })
                elif out == 'max':
                    r2_temp.update({
                        'r2_train_max': r2_train,
                        'r2_validation_max': r2_validation,
                        'r2_test_max': r2_test,
                        'r2_train_max_skl': r2_train_skl,
                        'r2_validation_max_skl': r2_validation_skl,
                        'r2_test_max_skl': r2_test_skl,
                        'rmse_train_max': rmse_train,
                        'rmse_validation_max': rmse_validation,
                        'rmse_test_max': rmse_test,
                        #'top_5_features_max': top_5_features
                    })

            # Append the row to the DataFrame
            r2_scores = pd.concat([r2_scores, pd.DataFrame([r2_temp])], axis=0, ignore_index=True)

#r2_scores.to_csv('ml_scripts/new_feature_selection/r2_scores.csv', index=False)

  0%|          | 0/10 [00:00<?, ?it/s]

(750,) (50, 15)
reflat: (750,)


ValueError: operands could not be broadcast together with shapes (750,) (50,15) 

In [9]:
r2_scores.to_csv('ml_scripts/new_feature_selection/r2_scores_and_features.csv', index=False)

### Read the csv to extract the features

In [18]:
#r2_scores = pd.read_csv('ml_scripts/new_feature_selection/r2_scores_and_features.csv')

top5_cdf = r2_scores[['seed', 'nvar', 'model', 'r2_train_cdf', 'r2_validation_cdf', 'r2_train_cdf_skl', 'r2_validation_cdf_skl', 'top_5_features_cdf']]
top5_max = r2_scores[['seed', 'nvar', 'model', 'r2_train_max', 'r2_validation_max', 'r2_train_max_skl', 'r2_validation_max_skl','top_5_features_max']]

r2_max_filtered = top5_max.groupby(['seed']).max()
r2_cdf_filtered = top5_cdf.groupby(['seed']).max()

r2_rf_cdf = top5_cdf[top5_cdf['model'] == 'random_forest']
r2_rf_cdf_filtered = r2_rf_cdf.groupby(['seed']).max()

r2_rf_max = top5_max[top5_max['model'] == 'random_forest']
r2_rf_max_filtered = r2_rf_max.groupby(['seed']).max()

TypeError: agg function failed [how->max,dtype->object]

In [4]:
r2_max_filtered

,nvar,model,r2_train_max,r2_validation_max,top_5_features_max
seed,,,,,
42,40,xgboost,0.852185,0.639819,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
1996,40,xgboost,0.678840,0.522026,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
2019,40,xgboost,0.754726,0.133433,"Index(['surface_pressure_mean_PCA_1', 'surface..."
43709,40,xgboost,0.729623,0.422502,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
45319,40,xgboost,0.995174,0.107603,"Index(['surface_pressure_mean_PCA_2', '10m_v_c..."
111194,40,xgboost,0.976980,0.369066,"Index(['mean_sea_level_pressure_mean_PCA_1',\n..."
15012025,40,xgboost,0.731132,0.399656,"Index(['surface_pressure_min_PCA_2', 'surface_..."
19052024,40,xgboost,0.688325,0.416109,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
19961106,40,xgboost,0.845333,0.209895,"Index(['mean_sea_level_pressure_mean_PCA_1', '..."


In [5]:
r2_cdf_filtered

,nvar,model,r2_train_cdf,r2_validation_cdf,top_5_features_cdf
seed,,,,,
42,40,xgboost,0.625322,0.615016,Index(['mean_surface_latent_heat_flux_std_PCA_...
1996,40,xgboost,0.665088,0.371755,"Index(['surface_pressure_mean_PCA_2',\n ..."
2019,40,xgboost,0.833816,-0.127812,"Index(['surface_pressure_mean_PCA_2',\n ..."
43709,40,xgboost,0.610295,0.200325,"Index(['mean_sea_level_pressure_std_PCA_2', 's..."
45319,40,xgboost,0.968800,-0.045752,"Index(['surface_pressure_mean_PCA_2', 'surface..."
111194,40,xgboost,0.992154,0.107956,Index(['mean_surface_latent_heat_flux_std_PCA_...
15012025,40,xgboost,0.840648,0.277385,Index(['mean_surface_sensible_heat_flux_std_PC...
19052024,40,xgboost,0.644961,0.307685,"Index(['mean_sea_level_pressure_min_PCA_1',\n ..."
19961106,40,xgboost,0.964940,0.126585,"Index(['mean_sea_level_pressure_mean_PCA_1', '..."


In [6]:
r2_rf_cdf_filtered

,nvar,model,r2_train_cdf,r2_validation_cdf,top_5_features_cdf
seed,,,,,
42,40,random_forest,0.507808,0.615016,"Index(['geopotential_1000_min_PCA_2', 'surface..."
1996,40,random_forest,0.663104,0.371755,"Index(['10m_u_component_of_wind_max_PCA_1',\n ..."
2019,40,random_forest,0.481017,-0.127812,"Index(['surface_pressure_mean_PCA_2',\n ..."
43709,40,random_forest,0.469107,0.200325,"Index(['10m_u_component_of_wind_max_PCA_1', 's..."
45319,40,random_forest,0.578514,-0.053951,"Index(['surface_pressure_mean_PCA_2', 'surface..."
111194,40,random_forest,0.664492,0.107956,"Index(['10m_u_component_of_wind_max_PCA_1', 'g..."
15012025,40,random_forest,0.458211,0.277385,"Index(['10m_u_component_of_wind_max_PCA_1', 's..."
19052024,40,random_forest,0.466754,0.307685,"Index(['mean_sea_level_pressure_min_PCA_1',\n ..."
19961106,40,random_forest,0.803911,0.126585,"Index(['10m_u_component_of_wind_max_PCA_1', 's..."


In [7]:
r2_rf_max_filtered

,nvar,model,r2_train_max,r2_validation_max,top_5_features_max
seed,,,,,
42,40,random_forest,0.650474,0.639819,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
1996,40,random_forest,0.658513,0.522026,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
2019,40,random_forest,0.754726,0.133433,"Index(['surface_pressure_mean_PCA_1', 'surface..."
43709,40,random_forest,0.729623,0.422502,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
45319,40,random_forest,0.604877,0.096877,"Index(['surface_pressure_mean_PCA_2', '10m_v_c..."
111194,40,random_forest,0.856437,0.369066,"Index(['10m_v_component_of_wind_max_PCA_2',\n ..."
15012025,40,random_forest,0.572886,0.399656,"Index(['surface_pressure_mean_PCA_1', 'surface..."
19052024,40,random_forest,0.666079,0.416109,"Index(['mean_sea_level_pressure_min_PCA_1', 's..."
19961106,40,random_forest,0.679251,0.209895,"Index(['10m_u_component_of_wind_max_PCA_1', 's..."


Index(['mean_sea_level_pressure_min_PCA_1', 'surface_pressure_mean_PCA_2',
       '10m_v_component_of_wind_max_PCA_2',
       'mean_surface_sensible_heat_flux_std_PCA_3',
       'mean_surface_sensible_heat_flux_min_PCA_1'],
      dtype='object')

### Extract the top features per output 

In [ ]:
test = r2_cdf_filtered['top_5_features_cdf'].apply(lambda x: x.replace("Index(", "").replace(", dtype='object')", "").replace("\n", "").replace("'", "").split(", "))
test

r2_cdf_filtered['top_5_features_cdf']
test2 = [item for sublist in r2_cdf_filtered['top_5_features_cdf'] for item in sublist]


In [100]:
r2_cdf_filtered['top_5_features_max']
#.apply(lambda x: list(x.values) if isinstance(x, pd.Index) else list(x)).apply(pd.Series)


seed
42          Index(['mean_sea_level_pressure_min_PCA_1', 's...
1996        Index(['mean_sea_level_pressure_min_PCA_1', 's...
2019        Index(['surface_pressure_mean_PCA_1', 'surface...
43709       Index(['mean_sea_level_pressure_min_PCA_1', 's...
45319       Index(['surface_pressure_mean_PCA_2', '10m_v_c...
111194      Index(['mean_sea_level_pressure_mean_PCA_1',\n...
15012025    Index(['surface_pressure_min_PCA_2', 'surface_...
19052024    Index(['mean_sea_level_pressure_min_PCA_1', 's...
19961106    Index(['mean_sea_level_pressure_mean_PCA_1', '...
28012025    Index(['mean_sea_level_pressure_min_PCA_1', 's...
Name: top_5_features_max, dtype: object